In [303]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import numpy as np
import tqdm

import cvxpy as cp

In [304]:
class Quantizer(nn.Module):
    def __init__(self, 
                 initial_codebook:torch.Tensor,
                 H: torch.Tensor,
                 Weights: torch.Tensor,
                 ):
        """Vector Quantizer

        Args:
            initial_codebook (torch.Tensor): the initial codebook (n,k)
            H (torch.Tensor): the Hessian of shape (n,n)
            Weights (torch.Tensor): weights of shape (n,n)
        """
        super(Quantizer, self).__init__()

        # self.codebook = nn.Parameter(initial_codebook, requires_grad=True)
        self.codebook = nn.Parameter(initial_codebook)
        self.H = H
        self.Weights = Weights
        n,k = initial_codebook.shape
        #randomly initialize the initial assignments through xavier uniform initialization
        #a matrix of shape (k,n)

        #logistic initialization
        initial_assignments = torch.empty(k,n,device = initial_codebook.device,
                                                            dtype = initial_codebook.dtype
                                                            ).uniform_(0,1)
        

        self.initial_assignments = nn.Parameter(#take the logistic of the initial assignments
                                                torch.log(initial_assignments / (1 - initial_assignments)),
                                                requires_grad = True
                                                )

        self.activation = nn.PReLU().to(initial_codebook.device)
        # self.initial_assignments = torch.empty(n,k,device = initial_codebook.device,
        #                                                     dtype = initial_codebook.dtype
        #                                                     ).uniform_(-1,1)

        print(self.H.dtype,self.Weights.dtype,self.codebook.dtype,self.initial_assignments.dtype)


    def sparse_penalty(self, assignments: torch.Tensor, target_sparsity:float, 
                       greater_penalty:float):
        """Penalty function for the assignments

        Args:
            assignments (torch.Tensor): the assignments of shape (k,n)
                assumed to be past through 
            target_sparsity (float): the target sparsity of the assignments, sparsity is column wise
            greater_penalty (float): the penalty for greater sparsity
            lesser_penalty (float): the penalty for lesser sparsity

        Returns:
            torch.Tensor: the penalty
        """
        sparsity = torch.sum(assignments, dim = 0) #shape (n,)
        sparsity = sparsity / assignments.shape[0]
        penalty = torch.relu(sparsity - target_sparsity)
        return torch.sum(greater_penalty * penalty), torch.sum(assignments)

    def penalty(self, assignments, beta, penalty_1_weight,
                penalty_2_weight):
        """Penalty function for the assignments

        Args:
            assignments (torch.Tensor): the assignments of shape (k,n)
                assumed to be past through 
            beta (float): a hyperparameter for the first penalty, this penalty
            is 1- |2*a_ij - 1|**(beta)

            penalty_1_weight (float): the weight of the first penalty
            penalty_2_weight (float): the weight of the second penalty
            this penalty is the sum of the assignments, therefore enforceing 
            sparsity

        Returns:
            torch.Tensor: the penalty
        """
        # assert torch.isfinite(torch.abs(2*assignments - 1)).all()
        penalty_1 = torch.sum(1 - torch.abs(2*assignments - 1)**beta)
        # print(penalty_1)
        # penalty_2, sparsity = self.sparse_penalty(assignments, 0.5, 1)
        # return penalty_1_weight * penalty_1 + penalty_2_weight * penalty_2, penalty_1, sparsity
        penalty_2 = torch.sum(assignments)
        # print(penalty_2)
        return penalty_1_weight * penalty_1 + penalty_2_weight * penalty_2, penalty_1, penalty_2
    
    def forward(self,beta,penalty_1_weight,
                penalty_2_weight):
        """Forward pass of the quantizer

        Args:
            beta (float): a hyperparameter for the first penalty, this penalty
            is 1- |2*a_ij - 1|**(beta)

            penalty_1_weight (float): the weight of the first penalty
            penalty_2_weight (float): the weight of the second penalty
            this penalty is the sum of the assignments, therefore enforceing 
            sparsity

        Returns:
            torch.Tensor: the quantized codebook
        """
        #get the assignments
        assignments = torch.sigmoid(self.initial_assignments)*1.2 - 0.1
        # print(assignments)
        # assert torch.isfinite(assignments).all()
        assignments = torch.clip(assignments,0,1)
        # assert torch.isfinite(assignments).all()

        #get the quantized_weights
        quantized_weights = self.codebook @ assignments
        assert torch.isfinite(quantized_weights).all()
        #get the difference between the quantized weights and the original weights
        diff = quantized_weights - self.Weights

        #get the loss
        loss = torch.einsum('ik,kl,il->', diff, self.H, diff)
        # assert torch.isclose(loss,torch.sum(diff**2))
        # loss = torch.sum(diff**2)
        #get the penalty
        penalty,p1,p2 = self.penalty(assignments, beta, penalty_1_weight, penalty_2_weight)
        # assert torch.isfinite(penalty)
        return loss + penalty, loss, penalty,p1,p2
    
    def get_rounded_error(self):

        #get the assignments
        assignments = torch.sigmoid(self.initial_assignments)*1.2 - 0.1
        assignments = torch.clip(assignments,0,1)
        assignments = torch.round(assignments)  
        #get the quantized_weights
        quantized_weights = self.codebook @ assignments

        #get the difference between the quantized weights and the original weights
        diff = quantized_weights - self.Weights

        #get the loss
        # loss = torch.einsum('ik,kl,il->', diff, self.H, diff)
        # assert torch.isclose(loss,torch.sum(diff**2))
        # loss = torch.sum(diff**2)
        loss = torch.einsum('ik,kl,il->', diff, self.H, diff)
        return loss
    
    def update_codebook(self):
        raise NotImplementedError
        with torch.no_grad():
            assignments = torch.sigmoid(self.initial_assignments)

            #cast to numpy
            assignments = assignments.cpu().numpy()
            weights = self.Weights.cpu().numpy()
            H = self.H.cpu().numpy()

            #use cvxpy to solve the optimization problem
            k,n = assignments.shape
            codebook = cp.Variable(self.codebook.shape)
            objective = cp.Minimize(cp.trace((weights - codebook @ assignments).T @ H @ (weights - codebook @ assignments)))
            constraints = []
            
            #no constraints
            prob = cp.Problem(objective, constraints)
            prob.solve()
            self.codebook = torch.tensor(codebook.value,device = self.codebook.device,dtype = self.codebook.dtype)
            
        return self.codebook

    def update_assignments(self):
        raise NotImplementedError
        #closed form update 
        with torch.no_grad():
            # assignments = self.initial_assignments
            # print(assignments.shape)
            #update formula for the assignments is
            #A = W^T C (C^T C)^-1

            codebook_inner_product = self.codebook.T @ self.H @ self.codebook
            inverse = torch.inverse(codebook_inner_product)

            self.initial_assignments = self.Weights @ self.H @ self.codebook @ inverse
            print(self.initial_assignments)
            assert torch.all(torch.isfinite(self.initial_assignments))
        
        #take the mean of the assignments
        mean = torch.mean(torch.abs(self.initial_assignments))
        #rescale the assignments
        self.initial_assignments = self.initial_assignments / mean
        #scale up the codebook
        self.codebook = self.codebook * mean



In [305]:
data = torch.load("/home/lliu/huffman/test/original_weights.pt")
weights = data['weights'].float().to(torch.device("cuda:6"))
H = data['H'].float().to(torch.device("cuda:6"))

In [ ]:
# def BetaScheduler_(i, T, scale, bias):

#     signal = (np.sin(((i%T)/T-0.5)*np.pi)+1)*0.5

#     return scale * signal + bias

# def BetaScheduler_(i, T, scale, bias):

#     signal = (np.sin(((i)/T)*np.pi)+1) - 1

#     return scale * signal + bias

def BetaScheduler_(i, T, scale, bias):

    # signal = (np.sin(((i%T)/T-0.5)*np.pi)+1)*0.5
    signal = (T-i)/T
    # signal = (np.cos(((i)/T)*np.pi*2))*0.5+0.5
    return scale * signal + bias

# def BetaScheduler_(i, T, scale, bias):

#     # signal = (np.sin(((i%T)/T-0.5)*np.pi)+1)*0.5
#     # signal = abs((i-T/2)*2/T)**5
#     signal = (i/T)**5
#     return scale * signal + bias

import matplotlib.pyplot as plt 

k = 256

torch.manual_seed(42)
torch.cuda.manual_seed(42)

#try to initalize the codebook from the eigenvectors of the weights
eigenvalues, eigenvectors = torch.linalg.eigh(weights)
print(eigenvalues.shape, eigenvectors.shape)
# eigenvectors = eigenvalues.vectors
indexs = torch.argsort(eigenvalues,descending=True)[:k]
#renormalize the eigenvectors
initial_codebook = eigenvectors[indexs,:].T / k
print(initial_codebook.shape)
quantizer = Quantizer(
                        # weights[indexs,:].T/k
                        initial_codebook
                    #   , torch.eye(H.shape[0],device = H.device,dtype = H.dtype)
                    , H
                      , weights)


n_iters = 10000
n_subiters = 10000
n_subiters_multiple = 1.1

beta_min, beta_max = 1, 4

penalty_1_weight = 1
penalty_2_weight = 1
penalty_2_multiple = 1 
 
beta = 5 
penalty_2_weight = 0.01

losses = []
rounded_errors = []
i_s = []

errors = []
penalties = []
binary_penalty = []
sparsity_penalty = []
betas = []
penalty_2_weights = []
lrs = []


optimizer = torch.optim.Adam([{'params': quantizer.codebook, 'lr': 5e-4},
                                {'params': quantizer.initial_assignments, 'lr': 1e-3}])
# scheduler_up = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)
# scheduler_down = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=1.005)

j = 0
prev_loss = float('inf')
patience_up = 25
patience_down = 25
impatience_1 = 0
impatience_2 = 0


for i in tqdm.tqdm(range(n_iters)):
    beta = BetaScheduler_(j, n_subiters, beta_max - beta_min, beta_min)
    # penalty_2_weight = BetaScheduler_(j, n_subiters*penalty_2_multiple, 0.2, 0.0)
    # print(beta)
    # print(beta)
    loss, loss1, loss2, p1, p2 = quantizer(beta, penalty_1_weight, penalty_2_weight)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # print(loss)

    assert torch.all(torch.isfinite(loss))

    losses.append(loss.item())
    errors.append(loss1.item())
    penalties.append(loss2.item())
    binary_penalty.append(p1.item())
    sparsity_penalty.append(p2.item())
    betas.append(beta)
    penalty_2_weights.append(penalty_2_weight)
    j+=1
    if i % 1000 == 0:
        # print(f"Loss: {loss.item()}, Error: {loss1.item()/torch.sum(torch.abs(weights**2))}, Loss2: {loss2.item()}, P1: {p1.item()}, sparsity: {p2.item()/(H.shape[0]*k)}, beta: {beta}, penalty_2_weight: {penalty_2_weight}")
        print(f"Loss: {loss.item()}, Error: {loss1.item()/H.shape[0]}, Loss2: {loss2.item()}, P1: {p1.item()}, sparsity: {p2.item()/(H.shape[0]*k)}, beta: {beta}, penalty_2_weight: {penalty_2_weight}")
        # wandb.log({"Loss": loss.item(), "Error": loss1.item()/H.shape[0], "Penalty": loss2.item(), "BinaryPenalty": p1.item(), "Sparsity": p2.item()/(H.shape[0]**2),
        #            "penalty_2_weight": penalty_2_weight, "beta": beta})
        fig = plt.figure()
        plt.hist(np.clip(torch.sigmoid(quantizer.initial_assignments).detach().cpu().numpy().flatten()*1.2-0.1,0,1), bins = 100,density = True)
        plt.xlim(0,1)
        plt.ylim(1e-5,1)
        plt.yscale('log')
        plt.title("Assignments")
        plt.savefig(f"test/assignments{i}.png")  
        plt.close(fig)  

        rounded_error = quantizer.get_rounded_error()
        rounded_errors.append(rounded_error.item())
        i_s.append(i)
        # print(f"Rounded Error: {rounded_error.item()/torch.sum(torch.abs(weights**2))}")
        print(f"Rounded Error: {rounded_error.item()/H.shape[0]}")
    if j % n_subiters == n_subiters - 1:
        n_subiters = int(n_subiters * n_subiters_multiple)
        j = 0
        impatience_1 = 0

    # if loss1.item() > prev_loss:
    #     impatience_1 += 1
    #     if impatience_1 > patience_up:
    #         scheduler_up.step()
    #         # print("Decreasing learning rate to ", scheduler_up.get_last_lr())
    #         impatience_1 = 0
    # if loss1.item() < prev_loss:
    #     impatience_2 += 1
    #     if impatience_2 > patience_down:
    #         scheduler_down.step()
    #         # print("Increasing learning rate to ", scheduler_down.get_last_lr())
    #         impatience_2 = 0 

    prev_loss = loss1.item()

    lrs.append(optimizer.param_groups[0]['lr'])

    

print(loss1.item()/H.shape[0], loss2.item(), p1.item(), p2.item()/(H.shape[0]**2))



In [ ]:
1/1.0001


In [ ]:
plt.plot(errors[100:])
plt.plot(i_s[1:],rounded_errors[1:])

In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.array(losses)/losses[0], label = "total loss")
plt.plot(np.array(errors)/errors[0], label = "error")
plt.plot(np.array(penalties)/penalties[0], label = "penalty")
plt.plot(np.array(binary_penalty)/binary_penalty[0], label = "binary penalty")
plt.plot(np.array(sparsity_penalty)/sparsity_penalty[0], label = "sparsity penalty")
plt.legend()
plt.yscale('log')

In [ ]:
plt.plot(np.array(sparsity_penalty)/(H.shape[0]*k))

In [ ]:
plt.plot(np.array(betas)/np.max(betas), label = "beta")
plt.plot(np.array(pe|nalty_2_weights)/np.max(penalty_2_weights), label = "penalty_2_weight")

In [ ]:
i = np.arange(1000)
T = 1000
plt.plot((np.cos(((i)/T)*np.pi*2))*0.5+0.5)
# (np.sin(((i)/T)*np.pi)+1)*0.5

In [ ]:
errors[-1]

In [ ]:
min(errors)/weights.shape[0]

In [ ]:
#get the assignments
assignments = torch.sigmoid(quantizer.initial_assignments)*1.2 - 0.1
assignments = torch.clip(assignments,0,1)
# assignments = torch.round(assignments)
print(assignments)
#round the assignments
# assignments = torch.round(assignments)
print(assignments)
#get the quantized_weights
quantized_weights = quantizer.codebook @ assignments
print(quantized_weights)
#get the difference between the quantized weights and the original weights
diff = quantized_weights - quantizer.Weights
print(quantizer.Weights)
print(torch.mean(torch.abs(diff**2))/torch.mean(torch.abs(quantizer.Weights**2)))
# print(diff)

In [ ]:
torch.einsum('ik,kl,il->', diff, quantizer.H, diff)/(quantizer.Weights.shape[0])


In [ ]:
import matplotlib.pyplot as plt
plt.imshow(torch.round(torch.sigmoid(quantizer.initial_assignments)).detach().cpu().numpy(),interpolation='nearest', aspect='auto')
#scale it to be a square image
#add a colorbar
plt.colorbar()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(torch.sigmoid(quantizer.initial_assignments).detach().cpu().numpy()*1.2-0.1,interpolation='nearest', aspect='auto')
#scale it to be a square image
#add a colorbar
plt.colorbar()
plt.show()


In [ ]:
torch.sum(assignments,dim = 0)/k

In [ ]:
plt.hist(assignments.detach().cpu().numpy().flatten(),bins=100)
# plt.yscale('log')

In [ ]:
a = assignments.detach().cpu().numpy().flatten()

plt.plot(np.sort(a), np.linspace(0,1,len(a)))

In [ ]:
import matplotlib.pyplot as plt
import glob


#create a gif from soem figures

import imageio

images = []
for fn in sorted(glob.glob("test/*.png")):
    images.append(imageio.imread(fn))
print(len(images))
imageio.mimsave('test.gif', images)


In [ ]:
print(quantizer.Weights)